# Long Sentence Resolution - Analyze Shortening Dataset Compression Rate

# Documentation

## What is a word?

* A word is any token that is not any of the following:
  * bracket
  * punctuation
  * quote
  * space
  * currency

## Connect Google Drive
 
* Run the cell "Connect Google Drive" and on the popup window click "Connect to Google Drive".
 
* In the new page opened, choose an account (your account) and click Allow.

## Configuration parameters

* **Preprocessed data**: The preprocessed data that will be loaded for creating the modeling dataset.

* **Preprocessed data full paths**: The full paths to use from the selected preprocessed data.

# Installations

In [ ]:
!pip install datasets==2.5.2 \
             spacy==3.4.1

# Imports

In [ ]:
import os, json

import numpy as np

import pandas as pd

import ipywidgets as widgets

import plotly.express as px

from pathlib import Path

from typing import List, Union, Dict

from datasets import load_from_disk

from google.colab import drive

from spacy.lang.en import English

from spacy.tokens import Doc, Token

# Connect Google Drive

In [ ]:
drive.mount('/content/drive', force_remount=True)

# Create paths

In [ ]:
PREPROCESSED_DATA_DIRECTORY_PATH = Path('/content/drive/My Drive/Pfizer - Long Sentence Resolution/Datasets/preprocessed')

# Functions / Classes

In [ ]:
class DocumentAnalysis:
  def __init__ (self):
    self._linguistics_model = English()

    self._linguistics_model.add_pipe("sentencizer")

  def extract_n_words_from_documents(self, documents: Union[List[str], np.ndarray, pd.Series]) -> np.ndarray:
    return np.array(list(map(self._extract_n_words_from_document, self._linguistics_model.pipe(documents, batch_size=50))))

  def _extract_n_words_from_document(self, doc: Doc) -> int:
    return sum(map(self._token_is_word, doc))

  def _token_is_word(self, token: Token) -> bool:
    return not any([ token.is_bracket,
                     token.is_punct,
                     token.is_currency,
                     token.is_quote,
                     token.is_space ]) # https://spacy.io/api/token#attributes

class WordBasedCompressionRate:
  def __init__ (self):
    self._document_analysis = DocumentAnalysis()

  def compute(self, uncompressed_documents: Union[List[str], np.ndarray, pd.Series], compressed_documents: Union[List[str], np.ndarray, pd.Series], use_aggregator: bool = True) -> Union[Dict, float]:
    assert(len(uncompressed_documents) == len(compressed_documents))

    uncompressed_n_words = self._document_analysis.extract_n_words_from_documents(uncompressed_documents)

    compressed_n_words = self._document_analysis.extract_n_words_from_documents(compressed_documents)

    compression_rates = compressed_n_words / uncompressed_n_words

    return np.mean(compression_rates) if use_aggregator else {
        'compression_rates': compression_rates,
        'uncompressed_n_words': uncompressed_n_words,
        'compressed_n_words': compressed_n_words
    }

def bin_value(value: float, bin_edges: List[float]) -> str:
  assert(len(bin_edges) >= 2)

  for i in range(1, len(bin_edges)):
    if value >= bin_edges[i-1] and value < bin_edges[i]:
      return f'[{bin_edges[i-1]}, {bin_edges[i]})'

  raise ValueError(f"The provided value '{value}' cannot be mapped to a bin.")

def generate_compression_rate_thresholds(start_value: float, end_value: float, n_of_samples: int) -> List[float]:
  return list(map(lambda o: np.round(o, 2), np.linspace(start_value, end_value, n_of_samples)))

def load_full_paths_from_preprocessed_data_path (data_path: Path) -> List[str]:
  with open(data_path / 'config.json') as f: return json.load(f) ['full_paths_in_preprocessed_data']

# Configuration parameters

In [ ]:
def on_preprocessed_data_dropdown_change(change):
  preprocessed_data_full_paths_select_multiple.options = preprocessed_data_full_paths_select_multiple.value = load_full_paths_from_preprocessed_data_path(PREPROCESSED_DATA_DIRECTORY_PATH / change.new)

preprocessed_data_dropdown = widgets.Dropdown(options=list(filter(lambda o: o.startswith('shortening_dataset'), os.listdir(PREPROCESSED_DATA_DIRECTORY_PATH))))

preprocessed_data_full_paths = load_full_paths_from_preprocessed_data_path(PREPROCESSED_DATA_DIRECTORY_PATH / preprocessed_data_dropdown.value)
preprocessed_data_full_paths_select_multiple = widgets.SelectMultiple(options=preprocessed_data_full_paths, value=preprocessed_data_full_paths, rows=10, layout={'width': '400px'})

preprocessed_data_dropdown.observe(on_preprocessed_data_dropdown_change, names='value')

display(widgets.HBox([widgets.Label(value="Preprocessed data:"), preprocessed_data_dropdown]))
display(widgets.HBox([widgets.Label(value="Preprocessed data full paths:"), preprocessed_data_full_paths_select_multiple]))

# Load preprocessed data

In [ ]:
%%time

preprocessed_data = load_from_disk(PREPROCESSED_DATA_DIRECTORY_PATH / preprocessed_data_dropdown.value)

preprocessed_data

# Filter preprocessed data

In [ ]:
filtered_data = preprocessed_data.filter(lambda o: o['full_path'] in preprocessed_data_full_paths_select_multiple.value)

filtered_data

# Select columns from preprocessed data

In [ ]:
filtered_data = filtered_data.remove_columns(set(filtered_data.column_names).difference(['source', 'target', 'dataset']))

filtered_data

# Convert dataset to dataframe

In [ ]:
filtered_data_df = filtered_data.to_pandas()

filtered_data_df

# Measure compression rate and #words in source and target

In [ ]:
word_based_compression_rate_metric = WordBasedCompressionRate()

In [ ]:
%%time

compression_rate_results = word_based_compression_rate_metric.compute(filtered_data_df['source'], filtered_data_df['target'], use_aggregator=False)

filtered_data_df['compression_rate'] = compression_rate_results['compression_rates']

filtered_data_df['source_n_words'] = compression_rate_results['uncompressed_n_words']

filtered_data_df['target_n_words'] = compression_rate_results['compressed_n_words']

filtered_data_df

# Statistics

## \#examples and %examples per dataset

In [ ]:
filtered_data_df['dataset'].value_counts()

In [ ]:
np.round((filtered_data_df['dataset'].value_counts(normalize=True) * 100), 2).astype(str) + '%'

## 5-number summaries for compression rate and #words in source and target per dataset

In [ ]:
filtered_data_df.groupby('dataset') [['source_n_words', 'target_n_words', 'compression_rate']].describe().apply(lambda s: s.apply('{0:.5f}'.format)).T

## Box plots for compression rate and #words in source and target per dataset (20% sampled)

In [ ]:
px.box(filtered_data_df.groupby('dataset', group_keys=False).apply(lambda x: x.sample(frac=0.2, random_state=42)), x="dataset", y="source_n_words", title='Box plot of source #words per dataset (20% sampled)')

In [ ]:
px.box(filtered_data_df.groupby('dataset', group_keys=False).apply(lambda x: x.sample(frac=0.2, random_state=42)), x="dataset", y="target_n_words", title='Box plot of target #words per dataset (20% sampled)')

In [ ]:
px.box(filtered_data_df.groupby('dataset', group_keys=False).apply(lambda x: x.sample(frac=0.2, random_state=42)), x="dataset", y="compression_rate", title='Box plot of compression rate per dataset (20% sampled)')

## Compression rate histogram

In [ ]:
%%time

compression_rate_thresholds = generate_compression_rate_thresholds(0.0, 1.0, 21)

bin_edges = compression_rate_thresholds + [np.inf]

filtered_data_df['compression_rate_bin'] = filtered_data_df['compression_rate'].apply(lambda o: bin_value(o, bin_edges))

filtered_data_df

### Histogram of compression rate on all data (50% sampled)

In [ ]:
fig = px.histogram(filtered_data_df.sample(frac=0.5, random_state=42), x="compression_rate_bin", category_orders=dict(compression_rate_bin=[ bin_value(o, bin_edges) for o in compression_rate_thresholds ]), title='Histogram of compression rate on all data (50% sampled)')

fig.update_layout(bargap=0.2)

fig.show()

### Histogram of compression rate per dataset (50% sampled)

In [ ]:
fig = px.histogram(filtered_data_df.sample(frac=0.5, random_state=42), x="compression_rate_bin", color="dataset", category_orders=dict(compression_rate_bin=[ bin_value(o, bin_edges) for o in compression_rate_thresholds ]), title='Histogram of compression rate per dataset (50% sampled)')

fig.update_layout(bargap=0.2)

fig.update_layout(hovermode="x unified")

fig.show()

## \#examples and %examples kept for at least X compression rate on all data

In [ ]:
stats_after_compression_rate_thresholding_df = pd.DataFrame({'compression_rate_threshold': [], 'n_of_examples': [], 'percentage_of_examples': [] })

for cr_threshold in compression_rate_thresholds:
  cr_filtered_data_df = filtered_data_df[filtered_data_df['compression_rate'] >= cr_threshold]

  stats_after_compression_rate_thresholding_df.loc[len(stats_after_compression_rate_thresholding_df.index)] = [cr_threshold, len(cr_filtered_data_df), np.round(len(cr_filtered_data_df) / len(filtered_data_df) * 100, 2) ]

stats_after_compression_rate_thresholding_df = stats_after_compression_rate_thresholding_df.astype({'compression_rate_threshold': float, 'n_of_examples': int, 'percentage_of_examples': float})

stats_after_compression_rate_thresholding_df.set_index('compression_rate_threshold', inplace=True)

stats_after_compression_rate_thresholding_df

In [ ]:
fig = px.line(stats_after_compression_rate_thresholding_df, x=stats_after_compression_rate_thresholding_df.index, y="percentage_of_examples", hover_data=stats_after_compression_rate_thresholding_df.columns, markers=True, title='%examples (and #examples) kept for at least X compression rate on all data')

fig.update_xaxes(tickvals=stats_after_compression_rate_thresholding_df.index)

fig.show()

## \#examples and %examples kept for at least X compression rate per dataset

In [ ]:
filtered_datasets = filtered_data_df['dataset'].unique()

stats_per_dataset_after_compression_rate_thresholding_df = pd.DataFrame({ 'compression_rate_threshold': [], 'dataset': [], 'n_of_examples': [], 'percentage_of_examples': [] })

for cr_threshold in compression_rate_thresholds:
  cr_filtered_data_df = filtered_data_df[filtered_data_df['compression_rate'] >= cr_threshold]

  cr_filtered_data_per_dataset_n_of_examples = cr_filtered_data_df['dataset'].value_counts()

  cr_filtered_data_per_dataset_percentage_of_examples = np.round(cr_filtered_data_df['dataset'].value_counts(normalize=True) * 100, 2)

  for dataset in filtered_datasets:
    stats_per_dataset_after_compression_rate_thresholding_df.loc[len(stats_per_dataset_after_compression_rate_thresholding_df.index)] = [ cr_threshold, dataset, cr_filtered_data_per_dataset_n_of_examples[dataset] if dataset in cr_filtered_data_per_dataset_n_of_examples else 0, cr_filtered_data_per_dataset_percentage_of_examples[dataset] if dataset in cr_filtered_data_per_dataset_percentage_of_examples else 0.0 ]

stats_per_dataset_after_compression_rate_thresholding_df = stats_per_dataset_after_compression_rate_thresholding_df.astype({'compression_rate_threshold': float, 'dataset': str, 'n_of_examples': int, 'percentage_of_examples': float})

stats_per_dataset_after_compression_rate_thresholding_df.set_index('compression_rate_threshold', inplace=True)

stats_per_dataset_after_compression_rate_thresholding_df.sort_values(by=['compression_rate_threshold', 'dataset'], inplace=True)

stats_per_dataset_after_compression_rate_thresholding_df

In [ ]:
fig = px.line(stats_per_dataset_after_compression_rate_thresholding_df, x=stats_per_dataset_after_compression_rate_thresholding_df.index, y="n_of_examples", color='dataset', hover_data=stats_per_dataset_after_compression_rate_thresholding_df.columns, markers=True, title='#examples (and %examples) kept for at least X compression rate per dataset')

fig.update_xaxes(tickvals=stats_per_dataset_after_compression_rate_thresholding_df.index)

fig.update_layout(hovermode="x unified")

fig.show()

In [ ]:
fig = px.line(stats_per_dataset_after_compression_rate_thresholding_df, x=stats_per_dataset_after_compression_rate_thresholding_df.index, y="percentage_of_examples", color='dataset', hover_data=stats_per_dataset_after_compression_rate_thresholding_df.columns, markers=True, title='%examples (and #examples) kept for at least X compression rate per dataset')

fig.update_xaxes(tickvals=stats_per_dataset_after_compression_rate_thresholding_df.index)

fig.update_layout(hovermode="x unified")

fig.show()